In [ ]:
import scipy.io as sio
import numpy as np

# Load the .mat files
mat_data4 = sio.loadmat('/content/audio_smir_single.mat')
mat_data3 = sio.loadmat('/content/audio_smir_single2.mat')

# Assuming the relevant data is stored in variables named 'audio_smir4' and 'audio_smir_data3' within the .mat files
# You might need to inspect the .mat files to confirm the exact variable names
audio_smir4 = mat_data4['audio_smir']
audio_smir_data3 = mat_data3['audio_smir']

# Make sure they’re the same length
L = min(audio_smir4.shape[1], audio_smir_data3.shape[1])
mix_smir = audio_smir4[:, :L] + audio_smir_data3[:, :L]

# Save the mixed data to a new .mat file
sio.savemat('audio_smir_mix.mat', {'mix_smir': mix_smir})

print("Audio files loaded, mixed, and saved successfully!")

Audio files loaded, mixed, and saved successfully!


In [ ]:
import scipy.io as sio
import soundfile as sf
import numpy as np

mat = sio.loadmat("audio_smir_mix.mat")
mix_smir = mat["mix_smir"]  # shape (Mics, L)

# Option A: average all mics to mono
mix_mono = mix_smir.mean(axis=0)

# Option B: just pick mic 1
# mix_mono = mix_smir[0,:]

sf.write("mixture.wav", mix_mono, 16000)   # assuming 16k Hz


In [ ]:
from scipy.signal import resample_poly

TARGET_SR = 8000
mixture_8k = resample_poly(mix_mono, TARGET_SR, 16000)
sf.write("mixture_8k.wav", mixture_8k, TARGET_SR)


In [ ]:
!pip install speechbrain
import torch
from speechbrain.inference.separation import SepformerSeparation as separator

sep_model = separator.from_hparams(
    source="speechbrain/sepformer-wsj02mix",
    savedir="pretrained_models/sepformer-wsj02mix",
    run_opts={"device": "cuda" if torch.cuda.is_available() else "cpu"}
)

mix_tensor = torch.tensor(mixture_8k).unsqueeze(0).float().to(sep_model.device)
est_sources = sep_model.separate_batch(mix_tensor)

out1 = est_sources[0,:,0].cpu().numpy()
out2 = est_sources[0,:,1].cpu().numpy()

sf.write("est_source1.wav", out1, TARGET_SR)
sf.write("est_source2.wav", out2, TARGET_SR)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 864.1/864.1 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 754.1/754.1 kB 44.0 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/speechbrain/utils/torch_audio_backend.py:57: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  available_backends = torchaudio.list_audio_backends()
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoi

hyperparams.yaml: 0.00B [00:00, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--sepformer-wsj02mix/snapshots/3a2826343a10e2d2e8a75f79aeab5ff3a2473531/hyperparams.yaml' -> '/content/pretrained_models/sepformer-wsj02mix/hyperparams.yaml'
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/sepformer-wsj02mix' if not cached
/usr/local/lib/python3.12/dist-packages/speechbrain/utils/torch_audio_backend.py:57: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  available_backends = torchaudio.list_audio_backends()
DEBUG:speechbrain.utils.parameter_tr

masknet.ckpt:   0%|          | 0.00/113M [00:00<?, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--sepformer-wsj02mix/snapshots/3a2826343a10e2d2e8a75f79aeab5ff3a2473531/masknet.ckpt' -> '/content/pretrained_models/sepformer-wsj02mix/masknet.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["masknet"] = /content/pretrained_models/sepformer-wsj02mix/masknet.ckpt
INFO:speechbrain.utils.fetching:Fetch encoder.ckpt: Fetching from HuggingFace Hub 'speechbrain/sepformer-wsj02mix' if not cached


encoder.ckpt:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--sepformer-wsj02mix/snapshots/3a2826343a10e2d2e8a75f79aeab5ff3a2473531/encoder.ckpt' -> '/content/pretrained_models/sepformer-wsj02mix/encoder.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["encoder"] = /content/pretrained_models/sepformer-wsj02mix/encoder.ckpt
INFO:speechbrain.utils.fetching:Fetch decoder.ckpt: Fetching from HuggingFace Hub 'speechbrain/sepformer-wsj02mix' if not cached


decoder.ckpt:   0%|          | 0.00/17.2k [00:00<?, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--sepformer-wsj02mix/snapshots/3a2826343a10e2d2e8a75f79aeab5ff3a2473531/decoder.ckpt' -> '/content/pretrained_models/sepformer-wsj02mix/decoder.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["decoder"] = /content/pretrained_models/sepformer-wsj02mix/decoder.ckpt
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: masknet, encoder, decoder
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): masknet -> /content/pretrained_models/sepformer-wsj02mix/masknet.ckpt
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): encoder -> /content/pretrained_models/sepformer-wsj02mix/encoder.ckpt
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): decoder -> /content/pretrained_models/sepformer-wsj02mix/decoder.ckpt


In [ ]:
from IPython.display import Audio, display

print("🎧 Original Mixture")
display(Audio(mixture_8k, rate=TARGET_SR))

print("🎧 Estimated Source 1")
display(Audio(out1, rate=TARGET_SR))

print("🎧 Estimated Source 2")
display(Audio(out2, rate=TARGET_SR))


🎧 Original Mixture


🎧 Estimated Source 1


🎧 Estimated Source 2


In [ ]:
!pip install -q openai-whisper jiwer

import whisper
from jiwer import wer

# Load ASR model
asr_model = whisper.load_model("medium")

import soundfile as sf

data1, sr1 = sf.read("/content/1673-143396-0005.flac")
data2, sr2 = sf.read("/content/1462-170138-0005.flac")

sf.write("source1.wav", data1, sr1)
sf.write("source2.wav", data2, sr2)

# Define transcription helper
def transcribe_audio(filepath):
    result = asr_model.transcribe(filepath, language="en")
    return result["text"]

# Transcribe separated sources
ref_text1 = transcribe_audio("source1.wav")   # reference transcription (from original mixture or ground truth)
ref_text2 = transcribe_audio("source2.wav")
hyp1 = transcribe_audio("est_source1.wav")
hyp2 = transcribe_audio("est_source2.wav")

print("Clean Source1:", ref_text1)
print("Clean Source2:", ref_text2)
print("Source1 transcription:", hyp1)
print("Source2 transcription:", hyp2)

# Compute WER
wer1 = wer(ref_text2, hyp1)
wer2 = wer(ref_text1, hyp2)

print(f"WER for Source1: {wer1:.2f}")
print(f"WER for Source2: {wer2:.2f}")


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Clean Source1:  The son of a virgin, generated by the ineffable operation of the Holy Spirit, was a creature without example or resemblance, superior in every attribute of mind and body to the children of Adam.
Clean Source2:  Do you know, Alexander?" Manehal looked with perplexity up into the top of the handsome and rubbed his pink cheek with his gloved finger. Do you know, I sometimes think of taking to criticism seriously myself.
Source1 transcription:  The son of a virgin, generated by the inethical operation of the Holy Spirit, was a creature without examples.
Source2 transcription:  Do you love Alexander? How would you possibly appreciate the power of the handgun?
WER for Source1: 0.94
WER for Source2: 0.91


########################################################################

In [ ]:
import scipy.io
import numpy as np
import torch
import torchaudio
from speechbrain.pretrained import SepformerSeparation as separator
from IPython.display import Audio, display
import whisper

# -----------------------------
# 1. Load .mat audio signals
# -----------------------------
mat1 = scipy.io.loadmat("audio_smir_single.mat")
mat2 = scipy.io.loadmat("audio_smir_single2.mat")

# The audio arrays might be stored with unknown key, find them:
def get_audio_from_mat(matfile):
    for k,v in matfile.items():
        if isinstance(v, np.ndarray) and v.ndim in [1,2]:
            return v.squeeze().astype(np.float32)
    return None

audio1 = get_audio_from_mat(mat1)
audio2 = get_audio_from_mat(mat2)

print("Audio1 length:", len(audio1))
print("Audio2 length:", len(audio2))

# Ensure same length
min_len = min(len(audio1), len(audio2))
audio1, audio2 = audio1[:min_len], audio2[:min_len]

# Mix them
mix = audio1 + audio2
sr = 16000  # assume 16kHz

# -----------------------------
# 2. Play Mixture
# -----------------------------
print("▶️ Mixture")
display(Audio(mix, rate=sr))

# -----------------------------
# 3. Separation using Sepformer
# -----------------------------
mix = mix.astype(np.float32).flatten()

# Convert to torch tensor [batch, time]
tmp_mix = torch.tensor(mix).unsqueeze(0)   # shape [1, T]

print("Input shape for Sepformer:", tmp_mix.shape)

# Load model
sep_model = separator.from_hparams(
    source="speechbrain/sepformer-wsj02mix",
    savedir="pretrained_models/sepformer-wsj02mix"
)

# Separate
est_sources = sep_model.separate_batch(tmp_mix)

# Convert to numpy
est1 = est_sources[0,0].detach().cpu().numpy()
est2 = est_sources[0,1].detach().cpu().numpy()

print("▶️ Separated Source 1")
display(Audio(est1, rate=sr))

print("▶️ Separated Source 2")
display(Audio(est2, rate=sr))

# -----------------------------
# 4. Transcription with Whisper
# -----------------------------
model = whisper.load_model("small")  # use "base" or "small" depending on Colab GPU

torchaudio.save("est1.wav", torch.tensor(est1).unsqueeze(0), sr)
torchaudio.save("est2.wav", torch.tensor(est2).unsqueeze(0), sr)

result1 = model.transcribe("est1.wav")
result2 = model.transcribe("est2.wav")

print("\nTranscription Source 1:", result1["text"])
print("Transcription Source 2:", result2["text"])


ModuleNotFoundError: No module named 'speechbrain'